# Step2 - Finetuning

In the second fine-tuning step (Step 2), we conducted our own SHAP analysis using battery data. The insights derived from this analysis were intended to be universally applicable to batteries and should be transferred to the LLM. The following notebook describes how this second unsupervised finetuning step was conducted.

## Start Training
To start the training, we have to restart the textgeneration-webui, using the following commands:

    conda activate textgen
    cd /home/[...]/text-generation-webui/
    python server.py --share --model models/Step1-model --load-in-8bit

Thereby, we load the selected base model (Step1-model) in 8bit as recommended by the text-generation-webui-documentation.

After the webui started, the following steps have been executed within the webui:

1. Switch to tab "Training"
2. Give the LoRA-file a new name - in our case "Step2_adapter"
3. Adapt the Hyperparameters for the training:
- Training-epochs: 3
- Learning-rate: 3e^-4
- LoRA Rank: 8
- LoRA Alpha: 16
- Batch Size: 128
- Micro Batch Size: 4
- Cutoff Length: 256
- LR Scheduler: linear
- Overlap Length: 128
- Prefer Newline Cut Length: 128
4. Go to the Tab "Raw text file"
5. Select under "Text file" the folder "Step2-Trainingdata"
6. Click "Start LoRA Training"

The new LoRA-adapter was saved to the folder "text-generation-webui/loras/Step2_adapter".

## Create Intermediate-Step2-model
We decided to create the new Intermediate-Step1-model by merging the newly trained LORA-adapter (Step2_adapter) with the corresponding base model (Step1-model). Therefore, we executed the following code from this notebook:

### Load Base-model, tokenizer, LoRA-adapter

In [5]:
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_path = "/home/jupyter-edt_wise34_tf2/text-generation-webui/models/Step1-model/"


model = AutoModelForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="offload",
    trust_remote_code=True
)
model.config.use_cache = False

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### Combine Base-model with LoRA-adapter

In [ ]:
peft_model = PeftModel.from_pretrained(model, "/home/jupyter-edt_wise34_tf2/text-generation-webui/loras/Step2_adapter/", is_trainable=True

### Merge Base-model with LoRA-adapter 

In [7]:
merged_model = peft_model.merge_and_unload()

### Save merged model to text-generation-webui

In [ ]:
merged_model.save_pretrained("/home/jupyter-edt_wise34_tf2/text-generation-webui/models/Step2-model/", safe_serializetion=True)
tokenizer.save_pretrained("/home/jupyter-edt_wise34_tf2/text-generation-webui/models/Step2-model/")

The new Step2-model is saved to the folder "/home/[...]/text-generation-webui/models".